<a href="https://colab.research.google.com/github/cicl-iscl/LeWiDi_SemEval2023/blob/main/Notebooks/MTL/STL_KL_loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install transformers
!pip install wandb

# STL approach
- Fine tune whole model on soft labels, with KL div as error
- Custom output layer with dropout
  - To test if custom output layer works as expected for more advanced MTL heads
- Potential Benchmark for my later MTL Models


In [2]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [54]:
from drive.MyDrive.cicl_data.helpers import read_data
# from drive.MyDrive.cicl_data.code import CustomLabelDataset

In [4]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
# from datasets import Dataset
from sklearn.metrics import f1_score
import torch.nn.functional as Fun
from torch.utils.data import Dataset, random_split, DataLoader
from torch.optim import AdamW
import torch.nn as nn

from tqdm.notebook import tqdm


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
data_dict = read_data()
df_all = pd.concat([data_dict[k] for k in data_dict.keys()])

### Pretrained model

In [7]:
# Maybe load from wandb in future
tokenizer = AutoTokenizer.from_pretrained("lanwuwei/GigaBERT-v4-Arabic-and-English", do_lower_case=True)

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/578 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/458k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/157 [00:00<?, ?B/s]

In [55]:
from transformers import BertModel
base_model = BertModel.from_pretrained("lanwuwei/GigaBERT-v4-Arabic-and-English")
# class_model = AutoModelForSequenceClassification.from_pretrained("lanwuwei/GigaBERT-v4-Arabic-and-English", num_labels=4)

# output of model: https://huggingface.co/docs/transformers/main_classes/output#transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions

Some weights of the model checkpoint at lanwuwei/GigaBERT-v4-Arabic-and-English were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [72]:
# TODO: Fit model for task
class SoftModel(nn.Module):
  def __init__(self, base_model):
    super().__init__()
    self.bert = base_model
    self.dropout = nn.Dropout(0.1)
    self.linear = nn.Linear(768, 2)
    # [8, 240, 768]

  def forward(self, input_ids, attention_mask, token_type_ids):
    """a linear layer on top of the pooled output (https://huggingface.co/transformers/v3.0.2/model_doc/bert.html#bertforsequenceclassification)"""
    x = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    # x = self.dropout(x.pooler_output)
    x = self.linear(x.pooler_output)
    return Fun.log_softmax(x, dim=1)

In [73]:
model = SoftModel(base_model).to(device)

## Prepare Data

In [11]:
class CustomLabelDataset(Dataset):
    def __init__(self, df_all):
        self.text = list(map(self.tokenize_func, df_all["text"]))
        self.soft_labels = df_all["soft_list"] 
        self.hard_labels = df_all["hard_label"]
        self.hard_labels_1h = Fun.one_hot(torch.tensor(df_all['hard_label'].values))

    def __len__(self):
        return len(self.text)
      
    def tokenize_func(self, text):
        return tokenizer(text, padding="max_length", truncation=True, max_length=240)

    def __getitem__(self, idx):
        input = {"attention_mask": torch.tensor(self.text[idx]["attention_mask"]),
                 "token_type_ids": torch.tensor(self.text[idx]["token_type_ids"]),
                 "input_ids": torch.tensor(self.text[idx]["input_ids"])}
        return input, self.hard_labels_1h[idx], torch.tensor(self.soft_labels[idx]), torch.tensor(self.hard_labels[idx])

In [12]:
# Init dataset
dataset = CustomLabelDataset(df_all)
batch_size = 4

train_size = int(len(dataset) * 0.8)
eval_size = len(dataset) - train_size
train_dataset, eval_dataset = random_split(dataset, [train_size, eval_size])

train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True)

eval_dataloader = DataLoader(
    eval_dataset,
    batch_size=batch_size)

## Optimization

In [74]:
# Optimizer
num_epochs = 2

num_training_steps = num_epochs * len(train_dataloader)
optimizer = AdamW(model.parameters())
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [59]:
# Loss
kl_loss = nn.KLDivLoss(reduction="batchmean")

## Training

In [61]:
run = wandb.init(
    project="mtl-1l",
    config={
        "epochs": num_epochs,
        "batch_size": batch_size,
        "device": device
        },
      save_code = True,
      tags = ["bert_arabic_english", "STL", "soft_labels", "Kl_loss", "dropout"],
      )
wandb.watch(model, log_freq=100)

[]

In [62]:
def ce_eval_func(model, eval_dataloader, eval_size, epsilon=1e-12):
  model.eval()
  cross_error = 0

  for i, batch in enumerate(tqdm(eval_dataloader, 0)):
    input_ids = batch[0]["input_ids"].to(device, dtype = torch.long)
    attention_mask = batch[0]["attention_mask"].to(device, dtype = torch.long)
    token_type_ids = batch[0]["token_type_ids"].to(device, dtype = torch.long)
    soft_labels = batch[2].to(device)

    with torch.no_grad():
      pred_sl = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    probabilities = torch.softmax(pred_sl, axis=-1)
    predictions = torch.clip(probabilities, epsilon, 1. - epsilon)
    cross_error += -torch.sum(soft_labels * torch.log(predictions + 1e-9))

  return cross_error / eval_size

In [75]:
# Train

for e in range(num_epochs):
  model.train()
  loss_batches = 0
  epoch_loss = 0
  epoch_len = len(train_dataloader)

  for i, batch in enumerate(train_dataloader):
    input_ids = batch[0]["input_ids"].to(device, dtype = torch.long)
    attention_mask = batch[0]["attention_mask"].to(device, dtype = torch.long)
    token_type_ids = batch[0]["token_type_ids"].to(device, dtype = torch.long)
    soft_labels = batch[2].to(device)

    # predict
    optimizer.zero_grad()
    pred_sl = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

    # loss
    loss = kl_loss(pred_sl, soft_labels)

    loss.backward()
    optimizer.step()
    lr_scheduler.step()

    # Log
    loss_batches += loss.item()
    epoch_loss += loss.item()
    wandb.log({"train/loss": loss.item()})

    log_n_batches = 200
    if i % log_n_batches == 0:
      if i == 0:
        log_n_batches = 1
      print(f"{e+1}: Last {log_n_batches} batches avg loss: {loss_batches/log_n_batches:>7f}  [{i}/{epoch_len}]")
      wandb.log({"train/loss_over_batches": loss_batches/log_n_batches})
      wandb.log({"train/epochs": e})
      loss_batches = 0
  
  epoch_loss /= i  # Not completely correct (Loss per batch but not every batch ahs same size)
  print(f"Epoch loss: {epoch_loss:>7f}  [{e+1}/{num_epochs}]")
  wandb.log({"train/epoch_loss": epoch_loss})

  # Eval error
  model.eval()
  ce = ce_eval_func(model, eval_dataloader, eval_size)
  print(f"Epoch CE evaluation: {ce:>6f}")
  wandb.log({"eval/epoch_ce": ce})
  

1: Last 1 batches avg loss: 0.206459  [0/2086]
1: Last 200 batches avg loss: 0.384851  [200/2086]
1: Last 200 batches avg loss: 0.390675  [400/2086]
1: Last 200 batches avg loss: 0.396786  [600/2086]
1: Last 200 batches avg loss: 0.394004  [800/2086]
1: Last 200 batches avg loss: 0.378215  [1000/2086]
1: Last 200 batches avg loss: 0.371380  [1200/2086]
1: Last 200 batches avg loss: 0.356474  [1400/2086]
1: Last 200 batches avg loss: 0.345410  [1600/2086]
1: Last 200 batches avg loss: 0.352326  [1800/2086]
1: Last 200 batches avg loss: 0.378483  [2000/2086]
Epoch loss: 0.374845  [1/2]
Epoch CE evaluation: 0.644942581653595
2: Last 1 batches avg loss: 0.463189  [0/2086]
2: Last 200 batches avg loss: 0.372703  [200/2086]
2: Last 200 batches avg loss: 0.352360  [400/2086]
2: Last 200 batches avg loss: 0.355873  [600/2086]
2: Last 200 batches avg loss: 0.357364  [800/2086]
2: Last 200 batches avg loss: 0.349619  [1000/2086]
2: Last 200 batches avg loss: 0.348902  [1200/2086]
2: Last 200 bat

Model dependent improvements:


## Evaluation
Todo:

In [76]:
# initialize Evaluation dataset
data_dict_dev = read_data("dev")
df_dev = pd.concat([data_dict_dev[k] for k in data_dict_dev.keys()])

dev_dataset = CustomLabelDataset(df_dev)
dev_batch_size = 4
dev_size = len(dev_dataset)

dev_dataloader = DataLoader(
    dev_dataset,
    batch_size=dev_batch_size)

In [93]:
# Cross Entropy Error
cross_error = ce_eval_func(model, dev_dataloader, dev_size)
print(f"CE error: {cross_error}")
# wandb.log({"dev/ce": cross_error})


CE error: 0.5933617353439331


In [82]:
def f1_eval_func(model, eval_dataloader, eval_size):
  model.eval()
  f1_error = 0

  for i, batch in enumerate(tqdm(eval_dataloader, 0)):
    input_ids = batch[0]["input_ids"].to(device, dtype = torch.long)
    attention_mask = batch[0]["attention_mask"].to(device, dtype = torch.long)
    token_type_ids = batch[0]["token_type_ids"].to(device, dtype = torch.long)
    hard_labels = batch[3].to(device)

    with torch.no_grad():
      pred = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    f1_error += f1_score(hard_labels.cpu(), pred.argmax(1).cpu(), average='micro')

  return f1_error / eval_size

In [83]:
# F1 micro Error
# This model predicts only soft labels

f1 = f1_eval_func(model, dev_dataloader, dev_size)
print(f"F1 error: {f1}")
wandb.log({"dev/f1": f1})

  0%|          | 0/557 [00:00<?, ?it/s]

F1 error: 0.18325842696629213


In [85]:
model.eval()
cross_error = 0
epsilon = 1e-12
for i, batch in enumerate(eval_dataloader):
  input_ids = batch[0]["input_ids"].to(device, dtype = torch.long)
  attention_mask = batch[0]["attention_mask"].to(device, dtype = torch.long)
  token_type_ids = batch[0]["token_type_ids"].to(device, dtype = torch.long)
  soft_labels = batch[2].to(device)

  with torch.no_grad():
    pred = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
  probabilities = torch.softmax(pred, axis=-1)
  predictions = torch.clip(probabilities, epsilon, 1. - epsilon)
  cross_error += -torch.sum(soft_labels * torch.log(predictions + 1e-9))
  break



In [91]:
print(predictions)
print(soft_labels)

tensor([[0.7344, 0.2656],
        [0.7344, 0.2656],
        [0.7344, 0.2656],
        [0.7344, 0.2656]], device='cuda:0')
tensor([[0.6000, 0.4000],
        [1.0000, 0.0000],
        [0.4000, 0.6000],
        [1.0000, 0.0000]], device='cuda:0')


### Finish

In [ ]:
# Save parameters

In [79]:
torch.save(model.parameters, 'model.pt')
artifact = wandb.Artifact(name='model_param', type='model')
artifact.add_file(local_path="model.pt")
run.log_artifact(artifact);

In [84]:
wandb.finish()

dev/ce,▁
dev/f1,▁
eval/epoch_ce,█▁
train/epoch_loss,█▁
train/epochs,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁███████████
train/loss,▇▆▇▆▄▂▁█████████████████████████████████
train/loss_over_batches,█▆█▇▄▁██████████████████████
dev/ce,0.58968
dev/f1,0.18326
eval/epoch_ce,0.58968
train/epoch_loss,0.34801


Model only predicts one distribution for all data points.
Maybe regularization can help, but it seems like a deeper problem, as MTL has the same problem.

Basically the same result as MTL with 2 heads (both cross Entropy loss)